version 1.0.2
#![ML Logo](https://raw.githubusercontent.com/chicochica10/utad-spark-ml/master/images/utad-spark-ml.1x_Banner_300.png)
# **Un motor de recomendación con Apache Spark**
## **Predicción de la clasificación (rating) de películas**
#### Uno de los casos de uso más común del big data es predecir que es lo que el usuario quiere. Esto permite a Google mostrarte anuncios (ads) relevantes, a Amazon recomendarte productos y a Netflix recomendarte películas que podrían gustarte. En este lab demostraremos como usar Apache Spark para recomendar películas a para un usuario. Empezaremos con técnicas básicas y utilizaremos la librería de machine learning de Spark [Spark MLlib][mllib] en concreto el algoritmo de Mínimos cuadradados alternados (Alternating Least Squares) para hacer prediciones más sofisticadas.
#### Utilizaremos un subset de 500.000 ratings de [movielens 10M stable benchmark rating dataset](http://grouplens.org/datasets/movielens/). El código sería el mismo para todo el dataset que tiene más de 21 millones de ratings.
#### En este lab:
#### *Parte 0*: Preliminares
#### *Parte 1*: Recomendaciones básicas
#### *Parte 2*: Filtros colaborativos
#### *Parte 3*: Prediciones personalizadas para ti
#### **Nota:** Cuidado con llamar a `collect()` en los datasets, si son pequeños no hay problema para ver como son los datos, pero si el dataset es grande no cabrán en la memoria donde se ejecuta el driver.
[mllib]: https://spark.apache.org/mllib/

### Código
#### El lab se puede completar usando Python básico y transformaciones y acciones de pySpark. La única librería necesaria es math. Con la excepción de las funciones de ML que veremos todos los ejercicios se pueden completar con operaciones utilizadas previamente.

In [5]:
import sys
import os
from test_helper import Test

baseDir = os.path.join('data')
inputPath = os.path.join('utad-spark', 'lab4', 'small')

ratingsFilename = os.path.join(baseDir, inputPath, 'ratings.dat.gz')
moviesFilename = os.path.join(baseDir, inputPath, 'movies.dat')

### **Parte 0: Preliminares**
#### Leeremos el fichero línea a línea y crearemos un RDD con la líneas parseadas.
#### Cada línea en el dataset de ratings (`ratings.dat.gz`) está formateada como:
####   `UserID::MovieID::Rating::Timestamp`
#### Cada línea en el dataset de películas (`movies.dat`) está formateada como:
####   `MovieID::Title::Genres`
#### El campo `Genres` tiene el siguiente formato:
####   `Genres1|Genres2|Genres3|...`
#### El formato de estos ficheros es uniforme y simple por lo que podemos utilizar [`split()`](https://docs.python.org/2/library/stdtypes.html#str.split) de Python para parsear las líneas.
#### El parseo de los dos ficheros devolverá dos RDDs.
* #### Para cada línea en el dataset de ratings crearemos una tupla de (UserID, MovieID, Rating). Descartamos el timestamp porque no lo necesitaremos para este ejercicio.
* #### Para cada línea en el dataset de pelis crearemos una tupla de (MovieID, Title). Descartamos Genres porque no lo necesitaremos para este ejercicio.

In [6]:
numPartitions = 2
rawRatings = sc.textFile(ratingsFilename).repartition(numPartitions)
rawMovies = sc.textFile(moviesFilename)

def get_ratings_tuple(entry):
    """ Parsea una línea en el dataset de ratings
    Args:
        entry (str): una linea en el dataset de ratings de la forma UserID::MovieID::Rating::Timestamp
    Returns:
        tuple: (UserID, MovieID, Rating)
    """
    items = entry.split('::')
    return int(items[0]), int(items[1]), float(items[2])


def get_movie_tuple(entry):
    """ Parsea una línea del dataset de pelis
    Args:
        entry (str): una línea en el dataset de pelis de la forma of MovieID::Title::Genres
    Returns:
        tuple: (MovieID, Title)
    """
    items = entry.split('::')
    return int(items[0]), items[1]


ratingsRDD = rawRatings.map(get_ratings_tuple).cache() #RDD [(UserID, MovieID, Rating)]
moviesRDD = rawMovies.map(get_movie_tuple).cache() #RDD [(MovieID, Title)]

ratingsCount = ratingsRDD.count()
moviesCount = moviesRDD.count()

print 'Hay %s ratings y %s pelis en los datasets' % (ratingsCount, moviesCount)
print 'Ratings: %s' % ratingsRDD.take(3)
print 'Pelis: %s' % moviesRDD.take(3)

assert ratingsCount == 487650
assert moviesCount == 3883
assert moviesRDD.filter(lambda (id, title): title == 'Toy Story (1995)').count() == 1
assert (ratingsRDD.takeOrdered(1, key=lambda (user, movie, rating): movie)
        == [(1, 1, 5.0)])

Hay 487650 ratings y 3883 pelis en los datasets
Ratings: [(1, 1193, 5.0), (1, 914, 3.0), (1, 2355, 5.0)]
Pelis: [(1, u'Toy Story (1995)'), (2, u'Jumanji (1995)'), (3, u'Grumpier Old Men (1995)')]


#### En este lab examinaremos subsets de las tuplas que hemos creado (por ejemplo, las pelis mejor calificadas por los usuarios). Ya que sólo es un subset de una dataset grande el resultado puede depender del orden en que se realicen las operaciones como los joins o como se hayan particionado los datos por los workers. Para garantizar que siempre se mostrarán los mismos resultados para un subset independientemente de como se hayan manipulado o almacenado los datos podemos ordenar el subset antes de examinarlo. La elección más obvia puede ser utilizar [`sortByKey()`][sortbykey] pero esta elección puede ser problemática ya que puede dar diferentes resultados si la key no es única.
#### Nota: es importante utilizar el [ tipo `unicode`](https://docs.python.org/2/howto/unicode.html#the-unicode-type) en vez del tipo `string` ya que los títulos están en codificados en unicode.
#### Considera el siguiente ejemplo, mientras que que los conjuntos son iguales, las listas impresas están normalmente en orden de valor diferente *aunque pueden casar de vez en cuando*
#### Puedes probar a ejecutar varias veces. Si falla el último assert no te preocupes que puede pasar.
[sortbykey]: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortByKey

In [7]:
tmp1 = [(1, u'alpha'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'delta')]
tmp2 = [(1, u'delta'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'alpha')]

oneRDD = sc.parallelize(tmp1)
twoRDD = sc.parallelize(tmp2)
oneSorted = oneRDD.sortByKey(True).collect()
twoSorted = twoRDD.sortByKey(True).collect()
print oneSorted
print twoSorted
assert set(oneSorted) == set(twoSorted)     # ambas listas tienen los mismos elementos
assert twoSorted[0][0] < twoSorted.pop()[0] # comprobar que están ordenadas por key
assert oneSorted[0:2] != twoSorted[0:2]     # el subset de los dos primeros elementos no casa

[(1, u'alpha'), (1, u'epsilon'), (1, u'delta'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]
[(1, u'delta'), (1, u'epsilon'), (1, u'alpha'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]


#### Aunque  las dos listas contengan tuplas idénticas la diferencia en la ordenacion *a veces* devuelve una ordenación diferente para el RDD ordenado (prueba a correr la celda repetidamente y ver si cambia el resultado o falla el último assert). Si sólo examinamos los dos primeros elementos del RDD (por ejemplo usando `take(2)`), podríamos observar respuestas diferentes **esto es una mala salida ya que queremos que entradas identicas produzcan resultados idénticos**. Una técnica de ordenación mejor es ordenar un RDD *por clave y valor* lo que podemos conseguir combinando ambos en una única string y ordenándola. Ya que la key es un entero y el valor una string unicode, usaremos una función que combine ambas en una única string unicode (por ejemplo `unicode('%.3f' % key) + ' ' + value`) antes de ordenar el RDD usando [sortBy()][sortby].
[sortby]: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy

In [8]:
def sortFunction(tuple):
    """ construye una Strig para ordenación (no realiza una ordenación real) 
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: el valor a ordenar: 'rating MovieName'
    """
    key = unicode('%.3f' % tuple[0])
    value = tuple[1]
    return (key + ' ' + value)


print oneRDD.sortBy(sortFunction, True).collect()
print twoRDD.sortBy(sortFunction, True).collect()

[(1, u'alpha'), (1, u'delta'), (1, u'epsilon'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]
[(1, u'alpha'), (1, u'delta'), (1, u'epsilon'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]


#### Si solo queremos ver los primeros elementos del RDD de manera ordenada podemos usar el método [takeOrdered] con la función `sortFunction` que hemos definido.
[takeordered]: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered

In [9]:
oneSorted1 = oneRDD.takeOrdered(oneRDD.count(),key=sortFunction)
twoSorted1 = twoRDD.takeOrdered(twoRDD.count(),key=sortFunction)
print 'one es %s' % oneSorted1
print 'two es %s' % twoSorted1
assert oneSorted1 == twoSorted1

one es [(1, u'alpha'), (1, u'delta'), (1, u'epsilon'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]
two es [(1, u'alpha'), (1, u'delta'), (1, u'epsilon'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]


### **Parte 1: Recomendaciones básicas**
#### Una forma de recomendar pelis es recomendar siempre las pelis con el rating medio más alto. En esta parte usaremos Spark para encontrar el nombre, número de ratings y la media de rating de las 20 pelis con el rating medio más alto y que tenga más de 500 críticas (reviews) . Vamos a eliminar las pelis con ratings altos pero con menos de (o igual a) 500 reviews porque pueden no tener suficiente atractivo para todo el mundo.

#### **(1a) Número de ratings y media de ratings para una peli**
#### Usando sólo Python, implementa una función auxiliar `getCountsAndAverages()` que tome una tupla de (MovieID, (Rating1, Rating2, Rating3, ...)) y devuelva una tupla de (MovieID, (number of ratings, averageRating)). Por ejemplo dada la tupla `(100, (10.0, 20.0, 30.0))`, debería devolver `(100, (3, 20.0))`

In [10]:
# TODO: Sustituye <RELLENA> con código apropiado

# Primero implementa la función auxiliar `getCountsAndAverages` usando sólo Python
def getCountsAndAverages(IDandRatingsTuple):
    """ Calcula el rating medio
    Args:
        IDandRatingsTuple: una única tupla de (MovieID, (Rating1, Rating2, Rating3, ...))
    Returns:
        tuple: una tupla de (MovieID, (number of ratings, averageRating))
    """
    movieId = IDandRatingsTuple[0]
    ratings = IDandRatingsTuple[1]
    ratingsSize = 0
    ratingsSum = 0
    for rating in ratings:
        ratingsSize = ratingsSize + 1
        ratingsSum = ratingsSum + rating
     
    return (int(movieId), (int(ratingsSize), float (ratingsSum)/ratingsSize))

In [11]:
# TEST Número de Ratings y media de Ratings para una peli (1a)

Test.assertEquals(getCountsAndAverages((1, (1, 2, 3, 4))), (1, (4, 2.5)),
                            'getCountsAndAverages() incorrecto con la lista de enteros')
Test.assertEquals(getCountsAndAverages((100, (10.0, 20.0, 30.0))), (100, (3, 20.0)),
                            'getCountsAndAverages() incorrecto con la lista de float')
Test.assertEquals(getCountsAndAverages((110, xrange(20))), (110, (20, 9.5)),
                            'getCountsAndAverages() incorrecto con xrange')

1 test passed.
1 test passed.
1 test passed.


#### **(1b) Pelis con la media de ratings más alta**
#### Ahora que podemos calcular la media de ratings, usaremos `getCountsAndAverages()` con Spark para determinar las pelis con la media de ratings más alta.
#### Los pasos a realizar son:
* #### Teniendo en cuenta que `ratingsRDD` contiene tuplas de la form (UserID, MovieID, Rating). Desde `ratingsRDD` crear un RDD con tuplas de la forma (MovieID, Python iterable de Ratings para ese MovieID). Esta transformación devolverá un RDD de la forma: `[(1, <pyspark.resultiterable.ResultIterable object at 0x7f16d50e7c90>), (2, <pyspark.resultiterable.ResultIterable object at 0x7f16d50e79d0>), (3, <pyspark.resultiterable.ResultIterable object at 0x7f16d50e7610>)]`. Sólo necesitaremos hacer dos transformaciones Spark para hacer este paso.
* #### Usando `movieIDsWithRatingsRDD` y `getCountsAndAverages()` calcula el número de ratings y la media de ratings para cada peli para devolver tuplas de la forma (MovieID, (number of ratings, average rating)). Ejemplo: `[(1, (993, 4.145015105740181)), (2, (332, 3.174698795180723)), (3, (299, 3.0468227424749164))]`. Puedes hacer esto con una única transformación de Spark
* #### Queremos ver los nombres de las pelis en vez de sus IDs. A `moviesRDD`, aplícale transformaciones que usen `movieIDsWithAvgRatingsRDD` para obtener los nombres de las pelis para `movieIDsWithAvgRatingsRDD`, devolviendo tuplas de la forma (media de rating, nombre de peli, número de ratings). Este conjunto de transformaciones devolverá un RDD de la forma `[(1.0, u'Autopsy (Macchie Solari) (1975)', 1), (1.0, u'Better Living (1998)', 1), (1.0, u'Big Squeeze, The (1996)', 3)]`. Necesitarás dos transformaciones de Spark para completar este paso: Primero usa `moviesRDD` con `movieIDsWithAvgRatingsRDD` para crear un nuevo RDD con los nombres de pelis que casen con los IDs de pelis y luego convierte el contenido de ese RDD en tuplas de (media de ratings, nombre de peli, número de rating). Estas transformaciones devolverán un RDD con el siguiente aspecto: `[(3.6818181818181817, u'Happiest Millionaire, The (1967)', 22), (3.0468227424749164, u'Grumpier Old Men (1995)', 299), (2.882978723404255, u'Hocus Pocus (1993)', 94)]`

In [12]:
# TODO: Sustituye <RELLENA> con código apropiado

# Para ratingsRDD con tuplas de (UserID, MovieID, Rating) crea un RDD con tuplas de
#  (MovieID, iterable de ratings para ese MovieID)
#RDD [(movieID, [rat1, rat2,...])]
movieIDsWithRatingsRDD = ratingsRDD.map (lambda (userId, movieId, rating): (movieId, rating)).groupByKey ()

print 'movieIDsWithRatingsRDD: %s\n' % movieIDsWithRatingsRDD.take(3)

# Usando `movieIDsWithRatingsRDD`, calcula el número de ratings y la media de ratings para cada peli
# para devolver tuplas de la forma (MovieID, (número de ratings, media de ratings))
#RDD [(MovieID, (número de ratings, media de ratings))]
movieIDsWithAvgRatingsRDD = movieIDsWithRatingsRDD.map (lambda IDandRatingsTuple: getCountsAndAverages(IDandRatingsTuple))

print 'movieIDsWithAvgRatingsRDD: %s\n' % movieIDsWithAvgRatingsRDD.take(3)

# A `movieIDsWithAvgRatingsRDD`, aplícale transformaciones RDD que usen `moviesRDD` para obtener el nombre
# de la peli para `movieIDsWithAvgRatingsRDD`, devolviendo tuplas de la forma
# (media de rating, nombre de la peli, número de ratings)

##moviesRDD : RDD [(MovieID, Title)]
movieNameWithAvgRatingsRDD = moviesRDD.join (movieIDsWithAvgRatingsRDD).map (lambda rec: (rec[1][1][1], rec[1][0], rec[1][1][0]))
#movieNameWithAvgRatingsRDD : RDD[(media de rating, nombre de la peli, número de ratings)]

print 'movieNameWithAvgRatingsRDD: %s\n' % movieNameWithAvgRatingsRDD.take(3)

movieIDsWithRatingsRDD: [(2, <pyspark.resultiterable.ResultIterable object at 0xb0ee79ec>), (4, <pyspark.resultiterable.ResultIterable object at 0xb0ee7b8c>), (6, <pyspark.resultiterable.ResultIterable object at 0xb0ee7a2c>)]

movieIDsWithAvgRatingsRDD: [(2, (332, 3.174698795180723)), (4, (71, 2.676056338028169)), (6, (442, 3.7918552036199094))]

movieNameWithAvgRatingsRDD: [(3.6818181818181817, u'Happiest Millionaire, The (1967)', 22), (3.0468227424749164, u'Grumpier Old Men (1995)', 299), (2.882978723404255, u'Hocus Pocus (1993)', 94)]



In [13]:
# TEST Pelis con la media de ratings más alta (1b)

Test.assertEquals(movieIDsWithRatingsRDD.count(), 3615,
                'movieIDsWithRatingsRDD.count() incorrecto (esperado 3615)')
movieIDsWithRatingsTakeOrdered = movieIDsWithRatingsRDD.takeOrdered(3)
Test.assertTrue(movieIDsWithRatingsTakeOrdered[0][0] == 1 and
                len(list(movieIDsWithRatingsTakeOrdered[0][1])) == 993,
                'count of ratings for movieIDsWithRatingsTakeOrdered[0] incorrecto (esperado 993)')
Test.assertTrue(movieIDsWithRatingsTakeOrdered[1][0] == 2 and
                len(list(movieIDsWithRatingsTakeOrdered[1][1])) == 332,
                'count of ratings for movieIDsWithRatingsTakeOrdered[1] incorrecto (esperado 332)')
Test.assertTrue(movieIDsWithRatingsTakeOrdered[2][0] == 3 and
                len(list(movieIDsWithRatingsTakeOrdered[2][1])) == 299,
                'count of ratings for movieIDsWithRatingsTakeOrdered[2] incorrecto (esperado 299)')

Test.assertEquals(movieIDsWithAvgRatingsRDD.count(), 3615,
                'movieIDsWithAvgRatingsRDD.count() incorrecto (esperado 3615)')
Test.assertEquals(movieIDsWithAvgRatingsRDD.takeOrdered(3),
                [(1, (993, 4.145015105740181)), (2, (332, 3.174698795180723)),
                 (3, (299, 3.0468227424749164))],
                'movieIDsWithAvgRatingsRDD.takeOrdered(3) incorrecto')

Test.assertEquals(movieNameWithAvgRatingsRDD.count(), 3615,
                'movieNameWithAvgRatingsRDD.count() incorrecto (esperado 3615)')
Test.assertEquals(movieNameWithAvgRatingsRDD.takeOrdered(3),
                [(1.0, u'Autopsy (Macchie Solari) (1975)', 1), (1.0, u'Better Living (1998)', 1),
                 (1.0, u'Big Squeeze, The (1996)', 3)],
                 'movieNameWithAvgRatingsRDD.takeOrdered(3) incorrecto')

1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.


#### **(1c) Pelis con las medias más altas de rating y más de 500 reviews**
#### Ahora que tenemos un RDD de pelis con los medias de ratings más altas podemos usar Spark para determinar las 20 pelis de ese RDD con más de 500 reviews.
#### Aplica una única transformación a `movieNameWithAvgRatingsRDD` para limitar el número de resultados de pelis con ratings de mas de 500 personas, después usa la función `sortFunction()` para ordenar por media de rating obteniendo las que tengan mayor rating primero. Tendrás un RDD de la siguiente forma: `[(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088), (4.515798462852263, u"Schindler's List (1993)", 1171), (4.512893982808023, u'Godfather, The (1972)', 1047)]`

In [14]:
# TODO: Sustituye <RELLENA> con código apropiado

# Aplica una transformación RDD a movieNameWithAvgRatingsRDD` para limitar los resultados de las pelis con
# ratings de más de 500 personas. Usa `sortFunction()` para ordenar por
# media de rating y obtener la pelis ordenada por su rating  (los ratings mayores primero)
#movieNameWithAvgRatingsRDD: RDD[(media de rating, nombre de la peli, número de ratings)]
movieLimitedAndSortedByRatingRDD = (movieNameWithAvgRatingsRDD
                                    .filter (lambda rec: rec[2] > 500)
                                    .sortBy(sortFunction, False))
print 'Pelis con los ratings más altos: %s' % movieLimitedAndSortedByRatingRDD.take(20)

Pelis con los ratings más altos: [(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088), (4.515798462852263, u"Schindler's List (1993)", 1171), (4.512893982808023, u'Godfather, The (1972)', 1047), (4.510460251046025, u'Raiders of the Lost Ark (1981)', 1195), (4.505415162454874, u'Usual Suspects, The (1995)', 831), (4.457256461232604, u'Rear Window (1954)', 503), (4.45468509984639, u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651), (4.43953006219765, u'Star Wars: Episode IV - A New Hope (1977)', 1447), (4.4, u'Sixth Sense, The (1999)', 1110), (4.394285714285714, u'North by Northwest (1959)', 700), (4.379506641366224, u'Citizen Kane (1941)', 527), (4.375, u'Casablanca (1942)', 776), (4.363975155279503, u'Godfather: Part II, The (1974)', 805), (4.358816276202219, u"One Flew Over the Cuckoo's Nest (1975)", 811), (4.358173076923077, u'Silence of the Lambs, The (1991)', 1248), (4.335826477187734, u'Saving Private Ryan (1998)', 1337), (4.3262411347

In [15]:
# TEST Pelis con la media de ratings más alta y mas de 500 reviews (1c)

Test.assertEquals(movieLimitedAndSortedByRatingRDD.count(), 194,
                'movieLimitedAndSortedByRatingRDD.count() incorrecto')
Test.assertEquals(movieLimitedAndSortedByRatingRDD.take(20),
              [(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088),
               (4.515798462852263, u"Schindler's List (1993)", 1171),
               (4.512893982808023, u'Godfather, The (1972)', 1047),
               (4.510460251046025, u'Raiders of the Lost Ark (1981)', 1195),
               (4.505415162454874, u'Usual Suspects, The (1995)', 831),
               (4.457256461232604, u'Rear Window (1954)', 503),
               (4.45468509984639, u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651),
               (4.43953006219765, u'Star Wars: Episode IV - A New Hope (1977)', 1447),
               (4.4, u'Sixth Sense, The (1999)', 1110), (4.394285714285714, u'North by Northwest (1959)', 700),
               (4.379506641366224, u'Citizen Kane (1941)', 527), (4.375, u'Casablanca (1942)', 776),
               (4.363975155279503, u'Godfather: Part II, The (1974)', 805),
               (4.358816276202219, u"One Flew Over the Cuckoo's Nest (1975)", 811),
               (4.358173076923077, u'Silence of the Lambs, The (1991)', 1248),
               (4.335826477187734, u'Saving Private Ryan (1998)', 1337),
               (4.326241134751773, u'Chinatown (1974)', 564),
               (4.325383304940375, u'Life Is Beautiful (La Vita \ufffd bella) (1997)', 587),
               (4.324110671936759, u'Monty Python and the Holy Grail (1974)', 759),
               (4.3096, u'Matrix, The (1999)', 1250)], 'incorrecto sortedByRatingRDD.take(20)')

1 test passed.
1 test passed.


#### Usando un threshold en el número de reviews podemos mejorar las recomendaciones pero hay otras formas de mejorar la calidad como por ejemplo ponderar los ratings por el número de ratings. 

## **Parte 2: Filtros Colaborativos**
#### Spark posee una librería de Machine Learning llamada  [MLlib][mllib].  En esta parte aprenderemos como utilizar MLib para hacer recomendaciones de pelis personalizadas utilizando los datos de las pelis.
#### Vamos a utilizar una técnica llamada [Filtro colaborativo (collaborative filtering)][collab]. El filtro colaborativo es un método para hacer prediciones automáticas (filtro) sobre los intereses de un usuario en base a las preferencias de otros usuarios (colaborativo). La idea de un filtro colaborativo es que si a una persona A que tiene la misma opinión que otra persona B respecto a algo, es más problable que la opinión de B en otro a otro asunto x sea igual que la de A que la de otra pesona elegida al azar. Puedes leer más sobre filtros colaborativos  [aquí][collab2].
#### La imagen de abajo ([Wikipedia][collab]) muestra un ejemplo de predicción de los rating de un usuario utilizando un filtro colaborativo. En primer lugar, la gente califica diferentes items (como vídeos, imagenes, juegos). Despúes el sistema hace prediciones sobre el rating de un usuario para un item que el usuario aún no ha calificado. Estas prediciones se construyen sobre los ratings existentes de otros usuarios que tienen ratings similares con el usuario activo. Por ejemplo en la imagen el sistema ha hecho la predición de que al usuario activo no le gustará el vídeo.
![collaborative filtering](https://courses.edx.org/c4x/BerkeleyX/CS100.1x/asset/Collaborative_filtering.gif)
[mllib]: https://spark.apache.org/mllib/
[collab]: https://en.wikipedia.org/?title=Collaborative_filtering
[collab2]: http://recommender-systems.org/collaborative-filtering/

#### Para recomendar peliculas empezaremos con una matriz cuyas entradas son ratings de usuarios (en rojo en el diagrama de abajo). Cada columna representa a un usuario (en verde) y cada fila representa una película en particular (en azul).
#### Ya que no todos los usuarios han calificado todas las pelis, no conocemos todas las entradas de la matriz que es precisamente por lo que necesitamos un filtro colaborativo. Para cada usuario tenemos ratings sólo para un subset de las pelis. Con el filtro colaborativo la idea es aproximar la matriz de ratings factorizada como el producto de dos matrices: una que describe las propiedades de cada usuario (en verde) y otra que describe las propiedades de cada peli (en azul)

![factorization](https://raw.githubusercontent.com/chicochica10/utad-spark-ml/master/images/matrix_factorization.png)
#### Queremos selecionar estas dos matrices de tal forma que el error para los pares usuarios/pelis sobre los que conocemos los ratings correctos este minimizado. El algoritmo de  [Mínimos cuadrados alternados (Alternating Least Squares)][als] hace esta tarea rellenando primero de manera aleatoria la matriz de usuarios con valores y después optimizando el valor de pelis para las que el error se minimiza. Depués fija la matriz de pelis para contrastar y optimiza el valor de la matriz de usuarios. Esta alternancia entre las matrices a optimizar es la razón por la que se usa el nombre de alternados.
#### Esta optimización es lo que se muestra a la derecha de la imagen de abajo. Dado un conjunto fijo de factores de usuario (por ejemplo los valores en la matriz de usuario) utilizamos los ratings conocidos para encontrar el mejor valor para los factores de película utilizando la optimización escrita abajo. Después "alternamos" y cogemos los mejores factores de usuario dejando fijos los factores de película
#### Para ver un ejemplo simple de como son las matrices de usuario y pelis revisa las transparencias.

[als]: https://en.wikiversity.org/wiki/Least-Squares_Method

#### **(2a) Creando un Conjunto de entrenamiento (Traininig Set)**
#### Antes de utilizar el algortimo de machine learning, necesitamos romper el dataset `ratingsRDD` en tres trozos:
* #### Un training set (RDD), que utilizaremos para entrenar modelos
* #### Un set de validación (RDD), que utilizaremos para elegir el mejor modelo
* #### Un set de test (RDD), que utilizaremos para nuestros experimentos
#### Para partir el dataset en múltiples grupos podemos utilizar pySpark [randomSplit()](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) . `randomSplit()` tomoa un conjunto de splits y devuelve multiples RDDs.

In [16]:
##ratings ->RDD [(UserID, MovieID, Rating)]
trainingRDD, validationRDD, testRDD = ratingsRDD.randomSplit([6, 2, 2], seed=0L)

print 'Training: %s, validación: %s, test: %s\n' % (trainingRDD.count(),
                                                    validationRDD.count(),
                                                    testRDD.count())
print trainingRDD.take(3)
print validationRDD.take(3)
print testRDD.take(3)

assert trainingRDD.count() == 292716
assert validationRDD.count() == 96902
assert testRDD.count() == 98032

assert trainingRDD.filter(lambda t: t == (1, 914, 3.0)).count() == 1
assert trainingRDD.filter(lambda t: t == (1, 2355, 5.0)).count() == 1
assert trainingRDD.filter(lambda t: t == (1, 595, 5.0)).count() == 1

assert validationRDD.filter(lambda t: t == (1, 1287, 5.0)).count() == 1
assert validationRDD.filter(lambda t: t == (1, 594, 4.0)).count() == 1
assert validationRDD.filter(lambda t: t == (1, 1270, 5.0)).count() == 1

assert testRDD.filter(lambda t: t == (1, 1193, 5.0)).count() == 1
assert testRDD.filter(lambda t: t == (1, 2398, 4.0)).count() == 1
assert testRDD.filter(lambda t: t == (1, 1035, 5.0)).count() == 1

Training: 292716, validación: 96902, test: 98032

[(1, 914, 3.0), (1, 2355, 5.0), (1, 595, 5.0)]
[(1, 1287, 5.0), (1, 594, 4.0), (1, 1270, 5.0)]
[(1, 1193, 5.0), (1, 2398, 4.0), (1, 1035, 5.0)]


#### Después de partir el dataset, el de entrenamiento tendrá unas 293.000 entradas y los de validación y test tendrán cada uno unas 97.000 (el número de entradas en cada dataset puede variar ligeramente debido a la naturaleza aleatoria de la transformación `randomSplit()`.

#### **(2b) Error Cuadrático medio - Root Mean Square Error (RMSE)**
#### En esta parte generaremos unos pocos modelos diferentes y necesitaremos una forma de decidir que modelo es el mejor. Utilizaremos  [Root Mean Square Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE)  o Desviación cuadrática media - Root Mean Square Deviation (RMSD) para calcular el error de cada modelo. RMSE se usa frecuentemente para medir diferencias entre valores (valores de una muestra y de toda la población) predicho por un modelo y los valores realmente observados. Las diferencias individuales se llaman residuos cuando los cálculos se realizan sobre la muestra de datos que es utilizada para la estimación y se llaman errores de predicción cuando se calculan para un dataset que no es de muestra. RMSE sirve para agregar las magnitudes de los errores en las predicciones en un proceso iterativo en una única medida de poder predictivo. RMSE es una buena medida de precisión para comparar errores de predicción de diferentes modelos para una variable particular pero no entre variables ya que depende de la escala.
#### El RMSE es la raíz cuadrada de la media de de (rating real - rating predicho) para todos los usuario y pelis para los que tenemos un rating real. A partir de la versión de Spark 1.4 existe el módulos [RegressionMetrics](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RegressionMetrics) que se puede utilizar para calcular RMSE. Pero para ver como hacerlo lo construiremos nosotros mismos
#### Vamos a escribir una función para calcular la suma de los errores cuadráticos sobre `predictedRDD` y `actualRDD` que tienen tuplas de la forma (UserID, MovieID, Rating)
#### Dados dos RDDs de ratings *x* e *y* con n tamaño *n* definimos RSME como: $ RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}$
#### Para calcular el RMSE, los pasos a realizar son:
* #### Transformar `predictedRDD` en tuplas ((UserID, MovieID), Rating). Por ejemplo tuplas como `[((1, 1), 5), ((1, 2), 3), ((1, 3), 4), ((2, 1), 3), ((2, 2), 2), ((2, 3), 4)]`. Puedes realizar este paso con una única transformación de Spark.
* #### Transformar `actualRDD` en tuplas ((UserID, MovieID), Rating). Por ejemplo tuplas como `[((1, 2), 3), ((1, 3), 5), ((2, 1), 5), ((2, 2), 1)]`. Puedes realizar este paso con una única transformación Spark.
* #### Utilizando únicamente transformaciones RDD (solo necesitaremos dos) calcular el error cuadrático para cada entrada que haga *matching* (esto es el mismo (UserID, MovieID) en cada RDD), en los RDDs transformados *no* uses `collect()`. No todo (UserID, MovieID) aparecerá en ambos RDDs, si un par no aparece en ambos no contribuye al RMSE. Al final terminaremos con un RDD con entradas de la forma  $ (x_i - y_i)^2$ Puedes ver en el módulo de Python [math](https://docs.python.org/2/library/math.html) como se calculan estos valores.

* #### Utilizando un acción RDD (**no** `collect()`), calcula el error cuadrático total $ SE = \sum_{i = 1}^{n} (x_i - y_i)^2 $
* #### Calcula *n* pero utilizando una acción RDD (que no sea `collect()` ) para contar el número de pares para los que se han calculado el error cuadrático total
* #### Usando el error cuadrático total y el número de pares, calcula el RSME. Asegúrate de calcular este valor como un [float](https://docs.python.org/2/library/stdtypes.html#numeric-types-int-float-long-complex).

In [17]:
# TODO: Sustituye <RELLENA> con código apropiado
import math

def computeError(predictedRDD, actualRDD):
    """ Calcula el error cuadrático medio entre los predichos y los reales
    Args:
        predictedRDD: ratings predichos para cada peli y cada usuario donde cada entrada tiene la forma
                      (UserID, MovieID, Rating)
        actualRDD: ratings reales donde cada entrada tiene la forma (UserID, MovieID, Rating)
    Returns:
        RSME (float): valor RSME calculado
    """
    # Transforma predictedRDD en tuplas ((UserID, MovieID), Rating)
    predictedReformattedRDD = predictedRDD.map (lambda (userID, movieID, rating): ((userID,movieID), rating))

    # Transforma actualRDD en tuplas ((UserID, MovieID), Rating)
    actualReformattedRDD = actualRDD.map (lambda (userID, movieID, rating): ((userID,movieID), rating))

    # Calcula el error cuadrático para cada entrada que haga matching: mismo (User ID, Movie ID) en cada
    # RDD) 
    squaredErrorsRDD = predictedReformattedRDD.join (actualReformattedRDD).map (lambda rec: (rec[1][0] - rec[1][1]) * (rec[1][0]-rec[1][1]))

    # Calcula el error cuadrático total
    totalError = squaredErrorsRDD.reduce (lambda a, b: a + b)

    # Cuenta el número de entradas para las que se ha calculado el error cuadrático total
    numRatings = squaredErrorsRDD.count()

    # calcula el RSME
    return math.sqrt(float(totalError) / numRatings)


# sc.parallelize nos devuelve una lista de Python en un RDD.
testPredicted = sc.parallelize([
    (1, 1, 5),
    (1, 2, 3),
    (1, 3, 4),
    (2, 1, 3),
    (2, 2, 2),
    (2, 3, 4)])
testActual = sc.parallelize([
     (1, 2, 3),
     (1, 3, 5),
     (2, 1, 5),
     (2, 2, 1)])
testPredicted2 = sc.parallelize([
     (2, 2, 5),
     (1, 2, 5)])
testError = computeError(testPredicted, testActual)
print 'Error para el dataset de test (debería ser 1.22474487139): %s' % testError

testError2 = computeError(testPredicted2, testActual)
print 'Error para el dataset2 de test (debería ser 3.16227766017): %s' % testError2

testError3 = computeError(testActual, testActual)
print 'Error para testActual dataset (debería ser 0.0): %s' % testError3

Error para el dataset de test (debería ser 1.22474487139): 1.22474487139
Error para el dataset2 de test (debería ser 3.16227766017): 3.16227766017
Error para testActual dataset (debería ser 0.0): 0.0


In [18]:
# TEST Root Mean Square Error (2b)
Test.assertTrue(abs(testError - 1.22474487139) < 0.00000001,
                'testError incorrecto (esperado 1.22474487139)')
Test.assertTrue(abs(testError2 - 3.16227766017) < 0.00000001,
                'testError2 incorrecto  result (esperado 3.16227766017)')
Test.assertTrue(abs(testError3 - 0.0) < 0.00000001,
                'testActual incorrecto result (esperado 0.0)')

1 test passed.
1 test passed.
1 test passed.


#### **(2c) Usando ALS.train()**
#### En esta parte usaremos la implementación de MLlib de mínimos cuadrados alternos  [ALS.train()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS). ALS toma un training dataset (RDD) y varios parámetros que controla el proceso de creación del modelo. Para determinar los mejores valores para los parámetros usaremos ALS para entrenar varios modelos, selecionaremos el mejor modelo y usaremos los parámetros de ese modelo en el resto de los ejercicios del lab.
#### El proceso que usaremos para determinar el mejor modelo sera:
* #### Seleccionar un conjunto de parámetros para el modelo. El parámetro más importante en `ALS.train()` es el *rank* que es el número de filas en la matriz de usuarios (en verde en el diagrama) o el numero de columnas en la matriz de pelis (azul en el diagrama). En general, un rank bajo significará errores grandes en el dataset de training, pero un rank alto puede pecar de [overfitting](https://en.wikipedia.org/wiki/Overfitting).) Usaremos ranks de 4, 8 y 12 utilizando el `trainingRDD` dataset.
* #### Crear un modelo utilizando `ALS.train(trainingRDD, rank, seed=seed, iterations=iterations, lambda_=regularizationParameter)` con tres parámetros: un RDD de tuplas de la forma (UserID, MovieID, rating) utilizado para entrenar el modelo, un rank de enteros (4, 8 ó 12), un número de iteraciones a ejecutar (usaremos 5 para el parámetro `iterations`) y un coeficiente de regularización (usaremos 0.1 para `regularizationParameter`).
* #### Para el paso de predición, crearemos un RDD de entrada `validationForPredictRDD`, formado por pares (UserID, MovieID) que se extraen de `validationRDD`. Al finalizar tendremos un RDD de la forma: `[(1, 1287), (1, 594), (1, 1270)]`
* #### Utilizando el modelo y `validationForPredictRDD`, podemos predecir rating llamando a [model.predictAll()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll) con el  `validationForPredictRDD` dataset, donde `model` es el modelo que se genera con ALS.train(). `predictAll` acepta un RDD con entradas de la forma (userID, movieID) y devuelve un RDD con cada entrada en la forma (userID, movieID, rating).
* #### Evaluaremos la calidad del modelo utilizando la función `computeError()` del apartado (2b) para calcular el error entre los ratings predichos y los ratings reales de `validationRDD`.
####  ¿Qué rank es el que produce el mejor modelo según RMSE con el dataset  `validationRDD`?
#### La operación llevará su tiempo (puedes observar el progreso con [Spark Web UI](http://localhost:4040). La mayor parte es para ejecutar la función `computeError()` ya que a diferencia de la implementación de ALS (y el módulo de Spark 1.4  [RegressionMetrics](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RegressionMetrics) ), no utiliza una librería rápida de álgebra linear y necesita ejecutar el código Phyton para todas las 100k entradas.

In [20]:
# TODO: Sustituye <RELLENA> con código apropiado
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDD.map (lambda (userId, movieId, rating) : (userId, movieId) )


seed = 5L
iterations = 5
regularizationParameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.03

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDD)
    errors[err] = error
    err += 1
    print 'para el rank %s el RMSE es %s' % (rank, error)
    if error < minError:
        minError = error
        bestRank = rank

print 'The best model was trained with rank %s' % bestRank

para el rank 4 el RMSE es 0.892734779484
para el rank 8 el RMSE es 0.890121292255
para el rank 12 el RMSE es 0.890216118367
The best model was trained with rank 8


In [21]:
# TEST Uso de ALS.train (2c)
Test.assertEquals(trainingRDD.getNumPartitions(), 2,
                  'número incorrecto de particiones para trainingRDD (se esperaban 2)')
Test.assertEquals(validationForPredictRDD.count(), 96902,
                  'tamaño incorrecto para validationForPredictRDD (se esperaba 96902)')
Test.assertEquals(validationForPredictRDD.filter(lambda t: t == (1, 1907)).count(), 1,
                  'valor incorrecto para validationForPredictRDD')
Test.assertTrue(abs(errors[0] - 0.883710109497) < tolerance, 'errors[0] incorrecto')
Test.assertTrue(abs(errors[1] - 0.878486305621) < tolerance, 'errors[1] incorrecto')
Test.assertTrue(abs(errors[2] - 0.876832795659) < tolerance, 'errors[2] incorrecto')

1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.


#### **(2d) Testeo del modelo**
#### Hemos usado los datasets `trainingRDD` y `validationRDD` para seleccionar el mejor model por lo que no podemos utilizarlos para saber como de bueno es nuestro model ya que podría ser muy vulnerable al [overfitting](https://en.wikipedia.org/wiki/Overfitting).  Es por eso que necesitamos usar el dataset `testRDD`. Utilizaremos el `bestRank` del apartado (2c) para crear un modelo para predecir los ratings para el dataset de test y calcular el RMSE.

#### Los pasos a dar son:
* #### Entrenar un modelo utilizando `trainingRDD`, `bestRank` y los parámetros que has usado en (2c): `seed=seed`, `iterations=iterations`, y `lambda_=regularizationParameter` - asegúrate que incluyes **todos** los parámetros.
* #### Para los pasos de predicción crea un RDD de entrada, `testForPredictingRDD`, con pares de la forma (UserID, MovieID) que extraigas de `testRDD`. Tendras que tener un RDD de la forma: `[(1, 1287), (1, 594), (1, 1270)]`
* #### Usa [myModel.predictAll()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll) para predecir los valores par el dataset de test.
* #### Para evaluar la calidad del modelo utiliza `testRDD` y la función `computeError` para calcular el error RMSE entre `testRDD` y `predictedTestRDD`.

In [23]:
# TODO: Sustituye <RELLENA> con código apropiado
myModel = ALS.train(trainingRDD, bestRank, seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)
testForPredictingRDD = testRDD.map (lambda (userId, movieId, rating): (userId, movieId))
predictedTestRDD = myModel.predictAll(testForPredictingRDD)


testRMSE = computeError(testRDD, predictedTestRDD)

print 'El modelo tiene un RMSE en el conjunto de test de %s' % testRMSE

El modelo tiene un RMSE en el conjunto de test de 0.891048561304


In [24]:
# TEST Testeo de tu modelo (2d)
Test.assertTrue(abs(testRMSE - 0.87809838344) < tolerance, 'testRMSE incorrecto')

1 test passed.


#### **(2e)Comparando el modelo**
#### Mirar el RMSE para los resultados predichos por el modelo vs los valores en el set de test es una forma de evaluar la calidad de nuestro modelo. Otra forma de evaluarlo es evaluar el error de un conjunto de test donde cada rating es la media para ese training set.
#### Los pasos a dar son:
* #### Usa `trainingRDD` para calcular la media de todas las pelis el dataset de training.
* #### Usa el rating medio que has determinado y el `testRDD` para crear un RDD con entradas de la forma  (userID, movieID, average rating).
* #### Usa la función `computeError` para calcular el RMSE entre el `testRDD` de validación que has creado y el  `testForAvgRDD`.

In [26]:
# TODO: Sustituye <RELLENA> con código apropiado

trainingAvgRating = trainingRDD.map (lambda (user, movie, rating):rating).mean()
print 'The average rating for movies in the training set is %s' % trainingAvgRating

testForAvgRDD = testRDD.map (lambda (user, movie, rating):(user,movie,trainingAvgRating))
testAvgRMSE = computeError(testRDD, testForAvgRDD)
print 'The RMSE on the average set is %s' % testAvgRMSE

The average rating for movies in the training set is 3.57409571052
The RMSE on the average set is 1.12036693569


In [27]:
# TEST Comparando el model (2e)
Test.assertTrue(abs(trainingAvgRating - 3.57409571052) < 0.000001,
                'trainingAvgRating incorrecto (esperado 3.57409571052)')
Test.assertTrue(abs(testAvgRMSE - 1.12036693569) < 0.000001,
                'testAvgRMSE incorrecto (esperado 1.12036693569)')

1 test passed.
1 test passed.


#### ¡Ahora ya podemos realizar prediciones de como los usuarios calificarán las pelis!

## **Part 3: Prediciones para ti**
#### Por último vamos a hacer recomendaciones para nosotros mismos. Para ello necesitamos añadir ratings personales al  `ratingsRDD` dataset.

#### **(3a) Tus ratings de pelis**
#### Para ayudarte a proporcionar ratings, tenemos el siguiente código que lista los nombres y los IDs de las 50 pelis con la calificaciones más altas de `movieLimitedAndSortedByRatingRDD` que creamos en la parte 1 del lab.

In [28]:
print 'Pelis más calificadas:'
print '(media de calificación, nombre de la peli, número de califiaciones)'
for ratingsTuple in movieLimitedAndSortedByRatingRDD.take(50):
    print ratingsTuple

Pelis más calificadas:
(media de calificación, nombre de la peli, número de califiaciones)
(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088)
(4.515798462852263, u"Schindler's List (1993)", 1171)
(4.512893982808023, u'Godfather, The (1972)', 1047)
(4.510460251046025, u'Raiders of the Lost Ark (1981)', 1195)
(4.505415162454874, u'Usual Suspects, The (1995)', 831)
(4.457256461232604, u'Rear Window (1954)', 503)
(4.45468509984639, u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651)
(4.43953006219765, u'Star Wars: Episode IV - A New Hope (1977)', 1447)
(4.4, u'Sixth Sense, The (1999)', 1110)
(4.394285714285714, u'North by Northwest (1959)', 700)
(4.379506641366224, u'Citizen Kane (1941)', 527)
(4.375, u'Casablanca (1942)', 776)
(4.363975155279503, u'Godfather: Part II, The (1974)', 805)
(4.358816276202219, u"One Flew Over the Cuckoo's Nest (1975)", 811)
(4.358173076923077, u'Silence of the Lambs, The (1991)', 1248)
(4.335826477187734, u'Saving 

#### El user ID 0 no está asignado, asi que lo usaremos para nuestros ratings utilizando la variable `myUserID`. A continuación creamos un nuevo RDD  `myRatingsRDD` con tus ratings para al menos 10 pelis. cada entrada debería estar formateada como `(myUserID, movieID, rating)` (de la misma manera que `trainingRDD`) y como el dataset original los ratings deberían estar entre 1 y 5 (incluidos). Si no has visto al menos 10 de las pelis del listado puedes incrementar el parámetro `take()` hasta que haya 10 películas que hayas visto.

In [29]:
# TODO: Sustituye <RELLENA> con código apropiado
myUserID = 0

# No confundas el rating con el movie ID!!!.
myRatedMovies = [
     (myUserID, 260, 5),(myUserID, 261, 4),(myUserID, 262, 3),(myUserID, 263, 3),(myUserID, 264, 2),(myUserID, 265, 1),
    (myUserID, 266, 5),(myUserID, 267, 4),(myUserID, 268, 3),(myUserID, 269, 1)

     # El formato de cada línea es (myUserID, movie ID, your rating)
     # Ejemplo "Star Wars: Episode IV - A New Hope (1977)" con un rating de 5 se añadiría como:
     #   (myUserID, 260, 5),
    ]
myRatingsRDD = sc.parallelize(myRatedMovies)
print 'ratings de mis pelis: %s' % myRatingsRDD.take(10)

ratings de mis pelis: [(0, 260, 5), (0, 261, 4), (0, 262, 3), (0, 263, 3), (0, 264, 2), (0, 265, 1), (0, 266, 5), (0, 267, 4), (0, 268, 3), (0, 269, 1)]


#### **(3b) Añade tus pelis al Training Dataset**
#### Ahor que tienes ratins para ti mismo, necesitas añadir tus ratings al dataset de `training` de tal forma que el modelo a entrenar incorpore tus preferencias. La transformación de Spark [union()](http://spark.apache.org/docs/latest/api/python/pyspark.rdd.RDD-class.html#union) combina dos RDDs. Utilizalá para crear un nuevo dataset de training con los datos originales y tus preferencias.


In [30]:
# TODO: Sustituye <RELLENA> con código apropiado
trainingWithMyRatingsRDD = trainingRDD.union (myRatingsRDD)

print ('The training dataset now has %s more entries than the original training dataset' %
       (trainingWithMyRatingsRDD.count() - trainingRDD.count()))
assert (trainingWithMyRatingsRDD.count() - trainingRDD.count()) == myRatingsRDD.count()

The training dataset now has 10 more entries than the original training dataset


#### **(3c) Entrenar un modelo con tus ratings**
#### Ahora entrena un modelo con tus ratings añadidos y los parámetros que usamos en la parte (2c): `bestRank`, `seed=seed`, `iterations=iterations`, y `lambda_=regularizationParameter` - asegúrate de que incluyes **todos** los parámetros.

In [32]:
# TODO: Sustituye <RELLENA> con código apropiado
myRatingsModel = ALS.train(trainingWithMyRatingsRDD, bestRank,  seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)

#### **(3d) Comprobar el RMSE para el nuevo modelo con tus ratings**
#### Calcular el RMSE para este nuevo modelo en el set de test.
* #### Para el paso de predicción reusamos `testForPredictingRDD`, con pares (UserID, MovieID) que hemos extraido de `testRDD`. El RDD tiene la forma `[(1, 1287), (1, 594), (1, 1270)]`
* #### Usa `myRatingsModel.predictAll()` para predecir los ratings para el dataset de test `testForPredictingRDD` y llámalo `predictedTestMyRatingsRDD`
* #### Para validar usa `testRDD` y la función `computeError` para calcular el RMSE entre `testRDD` y `predictedTestMyRatingsRDD` del modelo.

In [33]:
# TODO: Sustituye <RELLENA> con código apropiado
predictedTestMyRatingsRDD = myRatingsModel.predictAll (testForPredictingRDD)
testRMSEMyRatings = computeError (testRDD, predictedTestMyRatingsRDD)
print 'El modelo tiene un RMSE con el conjunto de test de %s' % testRMSEMyRatings

El modelo tiene un RMSE con el conjunto de test de 0.891995030395


#### **(3e) Predecir tus calificaciones **
#### Hasta el momento solo hemos utilizado el método  `predictAll` para calcular el error del modelo. Ahora vamos a usar `predictAll` para predecir que ratings darías a pelis que aún no has calificado.
#### Los pasos a dar son:
* #### Usa la lista de Python `myRatedMovies` para transformar `moviesRDD`en un RDD cuyas entradas son pares de la forma (myUserID, Movie ID) y no contienen ninguna película que hayas calificado. Esta transformación devolverá un RDD de la forma: `[(0, 1), (0, 2), (0, 3), (0, 4)]`. Puedes hacer este paso con una única transformación RDD.
* #### Para el paso de predición utiliza el RDD de entrada `myUnratedMoviesRDD` con myRatingsModel.predictAll() para predecir tus calificaciones para las pelis.

In [35]:
# TODO: Sustituye <RELLENA> con código apropiado

# Usa la lista de Python myRatedMovies para transformar moviesRDD en un RDD con entradas tipo (myUserID, Movie ID) 
# y que no contengasn ninguna peli que hayas calificado.
myUnratedMoviesRDD = (moviesRDD.map (lambda (movieId, movieTitle): (0,movieId)))

# usa myUnratedMoviesRDD con myRatingsModel.predictAll() para predecir los ratings sobre las pelis.
predictedRatingsRDD = myRatingsModel.predictAll (myUnratedMoviesRDD)


#### **(3f) Predecir tus calificaciones (2)**
#### Con los ratings predichos podemos imprimir las 25 pelis con la calificación más alta.
#### Los pasos a dar son:
* #### Por los apartados (1b) y (1c) sabemos que deberíamos sólo tener en cuenta pelis con un número razonable de reviews (por ejemplo, más de 75). Podemos experimentar con límites más bajos pero pocos ratings para una peli harán que tengamos más errores en la predicción. Transforma `movieIDsWithAvgRatingsRDD` del apartado (1b) que tiene la forma  (MovieID, (number of ratings, average rating)) en un RDD de la forma (MovieID, number of ratings): `[(2, 332), (4, 71), (6, 442)]`
* #### Queremos ver los nombres de las pelis en vez de los IDs, asi que transformaremos `predictedRatingsRDD` en un RDD con entradas de la forma (Movie ID, Predicted Rating): `[(3456, -0.5501005376936687), (1080, 1.5885892024487962), (320, -3.7952255522487865)]`
* #### Usa transformaciones RDD sobre `predictedRDD` y `movieCountsRDD` para devolver un un RDD con tuplas de la forma (Movie ID, (Predicted Rating, number of ratings)): `[(2050, (0.6694097486155939, 44)), (10, (5.29762541533513, 418)), (2060, (0.5055259373841172, 97))]`
* #### Usa transformaciones RDD sobre `predictedWithCountsRDD` y `moviesRDD` para devolver un RDD con tuplas de la forma (Predicted Rating, Movie Name, number of ratings), _para pelis con más de 75 ratings_ Por ejemplo: `[(7.983121900375243, u'Under Siege (1992)'), (7.9769201864261285, u'Fifth Element, The (1997)')]`

In [36]:
# TODO: Sustituye <RELLENA> con código apropiado

# Transforma movieIDsWithAvgRatingsRDD del apartado (1b), con la forma 
#(MovieID, (number of ratings, average rating)), en un RDD con la forma MovieID, number of ratings)
movieCountsRDD = movieIDsWithAvgRatingsRDD.map (lambda (movieId, (numbRatings, avgRatings)): (movieId, numbRatings))

# Transforma predictedRatingsRDD en un RDD cuyas entradas son pares (Movie ID, Predicted Rating)
predictedRDD = predictedRatingsRDD.map (lambda (userId, movieId, rating): (movieId, rating))

# Usa transformaciones RDD con predictedRDD y movieCountsRDD para devolver un RDD con tuplas de la forma
#(Movie ID, (Predicted Rating, number of ratings))
predictedWithCountsRDD  = predictedWithCountsRDD  = predictedRDD.join (movieCountsRDD)


# Usa transformaciones RDD con PredictedWithCountsRDD y moviesRDD para devolver un RDD con tuplas de la forma
#(Predicted Rating, Movie Name, number of ratings), para pelis con más de 75 ratings
ratingsWithNamesRDD = predictedWithCountsRDD.join (moviesRDD).filter (lambda (movieId,((rating,numRatings),title)): numRatings > 75)

predictedHighestRatedMovies = ratingsWithNamesRDD.takeOrdered(20, key=lambda x: -x[0])
print ('Mis pelis predichas con mayores calificaciones (para pelis con más de 75 reviews):\n%s' %
        '\n'.join(map(str, predictedHighestRatedMovies)))

Mis pelis predichas con mayores calificaciones (para pelis con más de 75 reviews):
(3952, ((3.29318936094958, 316), u'Contender, The (2000)'))
(3949, ((2.384084749668364, 214), u'Requiem for a Dream (2000)'))
(3948, ((3.4670776711437847, 665), u'Meet the Parents (2000)'))
(3946, ((1.4173383391462544, 83), u'Get Carter (2000)'))
(3937, ((2.5400325254619727, 115), u'Runaway (1984)'))
(3936, ((3.5786851172724727, 99), u'Phantom of the Opera, The (1943)'))
(3932, ((2.215342917195037, 202), u'Invisible Man, The (1933)'))
(3930, ((1.6745750714646945, 197), u'Creature From the Black Lagoon, The (1954)'))
(3929, ((2.801122225470703, 128), u'Bank Dick, The (1940)'))
(3928, ((2.655097852785038, 178), u'Abbott and Costello Meet Frankenstein (1948)'))
(3927, ((2.4762382438474844, 300), u'Fantastic Voyage (1966)'))
(3926, ((2.42046290776284, 154), u'Voyage to the Bottom of the Sea (1961)'))
(3925, ((1.246914226131255, 100), u'Stranger Than Paradise (1984)'))
(3923, ((1.2987791269204394, 91), u'Retu